In [1]:
import hashlib, json, sys


In [11]:
# Creating a hash fn to create a fingerprint for each transaction, linkin blocks together
# Defining a helper function to wrap the python hash fn used
def hashMe(msg=""):
    #helper function that wraps hashing alg.
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)
        
    if sys.version_info.major == 2:
        return unicode(haslib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()
    
                              

In [3]:
# want to create a fn to generate exchanges between Tyler and Conner
    # withdrawls = negative number , deposits = positive
#Constructing transactions to be between two users of our system and make sure deposits = withdrawls
    # not creating or destroying money

import random
random.seed(0)

def makeTransaction(maxValue = 5):
    #Will create valid transactions in range of (1,maxValue), maxValue = 5
    sign = int(random.getrandbits(1))*2 - 1 # Will randomly choose value -1 or 1
    amount = random.randint(1,maxValue)
    alicePays = sign * amount
    bobPays = -1 * alicePays
    # This construction will return transactions that respect conservation of tokens.
    # However, must check if transaction overdraft an account
    return {u'Alice':alicePays,u'Bob':bobPays}

In [4]:
# Now creating set of transactions, and chunk them into blocks
txnBuffer = [makeTransaction() for i in range(30)]

In [5]:
# Now to make blocks. Take first k transactions from buffer and turn into block, but first
# need to define a method for checking the validity of transactions we've pulled into the block
# Basic token system
    # deposits + withdrawls = 0     Tokens arent created or destroyed
    # Users account must have enough to pay withdrawls

def updateState(txn, state):
    #Inputs: txn, state: dictionaries keyed with account names that hold numeric values
    #for transfer amount (txn) or account balance (state)
    #Returns: Updated state, with additional users added to account balance (txn) if necessary
    #Note: Doesn't validate, updates transfer amount (txn)
    state = state.copy() #Working copy of data to avoid confusion of mutable dictionaries
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

In [8]:
def isValidTxn(txn,state):
    # Asumming transaction is a dictionary keyed by account names
    
    # Check that the sum of the deposits and withdrawls is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction is valid. Meaning doesnt cause over draft
    for key in txn.keys():
        if key in state.keys():
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) <0:
            return False
        
    return True

In [9]:
state = {u'Alice':5, u'Bob':5}

print(isValidTxn({u'Alice': -3, u'Bob': 3},state)) #Basic Transaction
print(isValidTxn({u'Alice': -4, u'Bob': 3},state)) # Doesn't work because destroying tokens
print(isValidTxn({u'Alice': -6, u'Bob': 6},state)) # OVerdraft, only have 5 tokens each
print(isValidTxn({u'Alice': -4, u'Bob': 2, u'Lisa': 2},state)) #creating users
print(isValidTxn({u'Alice': -5, u'Bob': 1, u'Lisa': 1, u'Conner': 2, u'Tyler': 1},state)) #Many new users

True
False
False
True
True


In [12]:
### Building the Blockchain: From Transactions to Blocks
    # Genesis Block: the first blook in the system
state = {u'Alice': 50, u'Bob':50}
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber' : 0, u'parentHash': None,u'txnCount': 1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash': genesisHash, u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys = True)

# Becomes first element from which everything else will be linked

In [13]:
chain = [genesisBlock]

In [14]:
# For each block, we want to collect set of transactions, create a header, hash it, and add to chain
def makeBlock(txns,chain):
    parentBlock = chain[-1]
    parentHash = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount = len(txns)
    blockContents = {u'blockNumber': blockNumber, u'parentHash': parentHash, u'txnCount':len(txns), u'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash': blockHash, u'contents':blockContents}
    
    return block

In [17]:
# using to process transaction buffer into a set of blocks

blockSizeLimit = 5 # Arbitary number of transactions per block, chosen by block miner and can vary between blocks

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn, state) # Will return False if txn is not valid
        
        if validTxn: # a valid state, not 'False'
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("Ignored Transaction : *Invalid*")
            sys.stdout.flush()
            continue # Says this was an invalid transaction, ignore and more on
            
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)
    
    

In [19]:
chain[0]

{'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]},
 'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507'}

In [20]:
chain[1]

{'contents': {'blockNumber': 1,
  'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': -4, 'Bob': 4},
   {'Alice': 1, 'Bob': -1},
   {'Alice': 5, 'Bob': -5},
   {'Alice': -1, 'Bob': 1},
   {'Alice': 5, 'Bob': -5}]},
 'hash': 'f6e176628034eb046c43e3091a8b8dd19e6f99ae9f9b13168387808dfd8f37aa'}

In [21]:
state

{'Alice': 77, 'Bob': 23}